In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import os
import random
import joblib
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

############################################
# 1. GLOBAL SETTINGS
############################################
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features = ["Price", "Open", "High", "Low"]
WINDOW_SIZE = 35
NUM_EPOCHS = 2000  # or 1000 if you prefer
BEST_MODELS_DIR = "."

############################################
# 2. Basic Data Preprocessing
############################################
def load_and_scale_data(train_csv, complete_csv):
    # Load partial CSV
    df_trainval = pd.read_csv(train_csv)
    df_trainval["Date"] = pd.to_datetime(df_trainval["Date"], errors="coerce")
    for col in ["Vol.", "Change %"]:
        if col in df_trainval.columns:
            df_trainval.drop(columns=[col], errors="ignore", inplace=True)

    for c in features:
        if c not in df_trainval.columns:
            df_trainval[c] = np.nan
        else:
            df_trainval[c] = df_trainval[c].astype(str).str.replace(",", "", regex=True)
            df_trainval[c] = pd.to_numeric(df_trainval[c], errors="coerce")

    df_trainval.sort_values("Date", inplace=True)
    df_trainval.reset_index(drop=True, inplace=True)

    # Basic cutoff
    train_cutoff = pd.to_datetime("2024-07-01")
    val_cutoff   = pd.to_datetime("2024-11-30")
    test_cutoff  = pd.to_datetime("2024-12-01")

    df_train = df_trainval[df_trainval["Date"] < train_cutoff].copy()
    df_val   = df_trainval[(df_trainval["Date"] >= train_cutoff) & (df_trainval["Date"] <= val_cutoff)].copy()
    df_test  = df_trainval[df_trainval["Date"] >= test_cutoff].copy()

    # Fit scaler on train only
    from sklearn.preprocessing import MinMaxScaler
    df_train_nonan = df_train.dropna(subset=features)
    scaler = MinMaxScaler()
    scaler.fit(df_train_nonan[features])

    # transform
    df_train_scaled = df_train.copy()
    if not df_train_nonan.empty:
        df_train_scaled.loc[df_train_nonan.index, features] = scaler.transform(df_train_nonan[features])

    df_val_scaled = df_val.copy()
    val_no_nan = df_val_scaled.dropna(subset=features)
    if not val_no_nan.empty:
        df_val_scaled.loc[val_no_nan.index, features] = scaler.transform(val_no_nan[features])

    df_test_scaled = df_test.copy()
    # fill missing with train set min
    train_mins = df_train_nonan[features].min()
    df_test_filled = df_test_scaled[features].fillna(train_mins)
    df_test_scaled.loc[:, features] = scaler.transform(df_test_filled)

    df_scaled = pd.concat([df_train_scaled, df_val_scaled, df_test_scaled], ignore_index=True)

    # Load complete CSV for final rolling
    df_complete = pd.read_csv(complete_csv)
    df_complete["Date"] = pd.to_datetime(df_complete["Date"], errors="coerce")
    for c in ["Vol.", "Change %"]:
        if c in df_complete.columns:
            df_complete.drop(columns=[c], inplace=True)
    for f in features:
        df_complete[f] = df_complete[f].astype(str).str.replace(",", "", regex=True).astype(float)
    df_complete.sort_values("Date", inplace=True)
    df_complete.reset_index(drop=True, inplace=True)

    return df_scaled, df_complete, scaler

############################################
# 3. Utility: Sequence & Slices
############################################
def make_sequences(df_subset, window=35):
    arr = df_subset[features].values
    dts = df_subset["Date"].values
    X_list, y_list, date_list = [], [], []
    for i in range(window, len(arr)):
        X_window = arr[i - window : i]
        y_target = arr[i]
        X_list.append(X_window)
        y_list.append(y_target)
        date_list.append(dts[i])
    return np.array(X_list), np.array(y_list), np.array(date_list)

############################################
# 4. Model Definitions
############################################
class BaseModel(nn.Module):
    def __init__(self, model_type="LSTM", window_width=35):
        super().__init__()
        self.model_type = model_type

        if model_type == "CNN":
            self.conv1 = nn.Conv1d(4, 64, 3)
            self.conv2 = nn.Conv1d(64, 128, 3)
            with torch.no_grad():
                dummy = torch.zeros(1, 4, window_width)
                outdummy = self.conv2(F.relu(self.conv1(dummy)))
                conv_output_size = outdummy.shape[1]*outdummy.shape[2]
            self.fc = nn.Linear(conv_output_size, 4)

        elif model_type == "LSTM":
            self.rnn = nn.LSTM(4, 128, num_layers=2, batch_first=True, dropout=0.05)
            self.fc = nn.Linear(128, 4)

        elif model_type == "GRU":
            self.rnn = nn.GRU(4, 128, num_layers=2, batch_first=True, dropout=0.05)
            self.fc = nn.Linear(128, 4)

        elif model_type == "RNN":
            self.rnn = nn.RNN(4, 128, num_layers=2, nonlinearity="relu", batch_first=True, dropout=0.05)
            self.fc = nn.Linear(128, 4)

        elif model_type == "EnhancedLSTM":
            self.rnn = nn.LSTM(4, 128, num_layers=3, batch_first=True, dropout=0.1)
            self.bn  = nn.BatchNorm1d(128)
            self.dropout = nn.Dropout(0.1)
            self.fc = nn.Linear(128, 4)

        elif model_type == "Transformer":
            self.input_linear = nn.Linear(4, 128)
            encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8, dropout=0.05)
            self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
            self.fc = nn.Linear(128,4)

        elif model_type in ["N-BEATS","N-HITS"]:
            self.input_size= window_width*4
            self.blocks= nn.ModuleList([nn.Sequential(
                nn.Linear(self.input_size,128),
                nn.ReLU(),
                nn.Linear(128,128),
                nn.ReLU(),
                nn.Linear(128,4)
            ) for _ in range(3)])

        else:
            raise ValueError(f"Invalid model_type: {model_type}")

    def forward(self, x):
        if self.model_type == "CNN":
            x = x.permute(0,2,1)
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x = x.view(x.size(0), -1)
            return self.fc(x)

        elif self.model_type in ["LSTM","GRU","RNN"]:
            out, _ = self.rnn(x)
            out = out[:,-1,:]
            return self.fc(out)

        elif self.model_type == "EnhancedLSTM":
            out, _= self.rnn(x)
            out = out[:,-1,:]
            out = self.bn(out)
            out = self.dropout(out)
            return self.fc(out)

        elif self.model_type == "Transformer":
            x = self.input_linear(x)
            x = x.permute(1,0,2)
            x = self.transformer_encoder(x)
            x = x[-1,:,:]
            return self.fc(x)

        elif self.model_type in ["N-BEATS","N-HITS"]:
            xflat= x.reshape(x.size(0), -1)
            forecast=0
            for block in self.blocks:
                forecast += block(xflat)
            return forecast

        else:
            raise ValueError("Invalid model_type")

class EarlyStopping:
    def __init__(self, patience=100, min_delta=0.0):
        self.patience= patience
        self.min_delta= min_delta
        self.counter= 0
        self.best_loss= None
        self.early_stop= False
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss> self.best_loss - self.min_delta:
            self.counter+=1
            if self.counter>= self.patience:
                print("Early stopping triggered")
                self.early_stop= True
        else:
            self.best_loss= val_loss
            self.counter=0

def inverse_transform_cols(arr4, scaler):
    return scaler.inverse_transform(arr4)

############################################
# 5. Validation Loss
############################################
def half_blind_validation_loss(model, df_scaled, val_start, val_end, window=35, scaler=None):
    """
    MSE over the range [val_start..val_end],
    using the last 'window' days for each day,
    not revealing future data.
    """
    import torch.nn as nn
    criterion= nn.MSELoss()
    df_work= df_scaled.copy()
    df_work.sort_values("Date", inplace=True)

    val_dates= np.sort(df_work.loc[(df_work["Date"]>= val_start)&(df_work["Date"]<= val_end),"Date"].unique())
    losses=[]
    model.eval()
    with torch.no_grad():
        for d in val_dates:
            idx= df_work.index[df_work["Date"]== d]
            if len(idx)==0:
                continue
            idx= idx[0]
            st= idx-window
            if st<0:
                continue
            window_df= df_work.iloc[st:idx][features].values
            if np.isnan(window_df).any():
                continue
            X_ = torch.tensor(window_df, dtype=torch.float).unsqueeze(0).to(device)
            pred_= model(X_).cpu()[0]
            realvals= df_work.loc[idx, features].values.astype(float)
            if np.isnan(realvals).any():
                continue
            lossval= criterion(pred_, torch.tensor(realvals).float())
            losses.append(lossval.item())

    if not losses:
        return 999999.0
    return np.mean(losses)

############################################
# 6. "Train or Load" PyTorch & sklearn
############################################
class SklearnWrapper:
    def __init__(self, model):
        self.model= model
    def forward(self, x):
        arr= x.cpu().numpy().reshape(1,-1)
        pred= self.model.predict(arr)
        return torch.from_numpy(pred).float().to(x.device)
    def eval(self):
        pass
    def __call__(self, x):
        return self.forward(x)

def train_or_load_torch(model_type, window_size, df_scaled_in, train_cutoff_in, val_cutoff_in,
                        model_save_name="best_model.pt", num_epochs=1500, scaler=None):
    import torch.nn as nn
    import torch.optim as optim

    if os.path.exists(model_save_name):
        print(f"[PyTorch] Loading existing {model_type} from {model_save_name}")
        net= BaseModel(model_type, window_size).to(device)
        net.load_state_dict(torch.load(model_save_name, map_location=device))
        net.eval()
        return net
    else:
        print(f"[PyTorch] Training {model_type} from scratch => {model_save_name}")
        df_local= df_scaled_in.copy()
        df_local.sort_values("Date", inplace=True)

        df_train_local= df_local[df_local["Date"]< train_cutoff_in].copy()
        df_val_local=   df_local[(df_local["Date"]>= train_cutoff_in)&(df_local["Date"]<= val_cutoff_in)].copy()
        # build sequences
        df_train_nonan= df_train_local.dropna(subset=features)
        if df_train_nonan.empty:
            raise ValueError("No data in rolling window for PyTorch training")
        X_,y_,_= make_sequences(df_train_nonan, window_size)
        X_t= torch.tensor(X_, dtype=torch.float)
        y_t= torch.tensor(y_, dtype=torch.float)
        from torch.utils.data import TensorDataset, DataLoader
        ds= TensorDataset(X_t,y_t)
        dl= DataLoader(ds, batch_size=len(ds), shuffle=False)

        net= BaseModel(model_type, window_size).to(device)
        criterion= nn.MSELoss()
        optimizer= optim.AdamW(net.parameters(), lr=0.002, weight_decay=0)

        stopper= EarlyStopping(patience=100)
        best_loss= 9999999.0
        for ep in range(num_epochs):
            net.train()
            for Xb, yb in dl:
                Xb, yb= Xb.to(device), yb.to(device)
                optimizer.zero_grad()
                out= net(Xb)
                loss= criterion(out,yb)
                loss.backward()
                optimizer.step()

            # val loss
            val_loss= half_blind_validation_loss(net, df_local, train_cutoff_in, val_cutoff_in, window=window_size)
            if ep>0 and ep%200==0:
                print(f"Epoch {ep}: val_loss={val_loss:.6f}")

            stopper(val_loss)
            if val_loss< best_loss:
                best_loss= val_loss
                torch.save(net.state_dict(), model_save_name)
            if stopper.early_stop:
                print(f"Early stop at epoch {ep}")
                break

        net.load_state_dict(torch.load(model_save_name, map_location=device))
        net.eval()
        return net

def train_or_load_sklearn(model_type, df_scaled_in, train_cutoff_in, val_cutoff_in,
                          model_save_name="best_sklearn.pkl", window_size=35, scaler=None):
    if os.path.exists(model_save_name):
        print(f"[sklearn] Loading {model_type} => {model_save_name}")
        loaded= joblib.load(model_save_name)
        return SklearnWrapper(loaded)
    else:
        print(f"[sklearn] Training {model_type} => {model_save_name}")
        df_work= df_scaled_in.copy()
        df_work.sort_values("Date", inplace=True)

        df_train_local= df_work[df_work["Date"]< train_cutoff_in].copy()
        df_val_local=   df_work[(df_work["Date"]>= train_cutoff_in)&(df_work["Date"]<= val_cutoff_in)].copy()
        df_train_nonan= df_train_local.dropna(subset=features)
        if df_train_nonan.empty:
            raise ValueError("No data in rolling window for sklearn train")

        X_, y_, _= make_sequences(df_train_nonan, window_size)
        Xf= X_.reshape(X_.shape[0], window_size*4)

        if model_type=="SVM":
            base= SVR(kernel="rbf",C=10, epsilon=0.01)
        elif model_type=="GPR":
            base= GaussianProcessRegressor(alpha=1e-2)
        elif model_type=="Boost":
            base= GradientBoostingRegressor(n_estimators=200, max_depth=3)
        else:
            raise ValueError("Unsupported model_type for sklearn wrapper")

        from sklearn.multioutput import MultiOutputRegressor
        mo= MultiOutputRegressor(base)
        mo.fit(Xf,y_)
        joblib.dump(mo, model_save_name)
        return SklearnWrapper(mo)

############################################
# 7. Full-Blind Forecast with Noise
############################################
def get_latest_window(df_actual, current_date, window=35, scaler=None):
    mask= df_actual["Date"]< current_date
    df_sub= df_actual.loc[mask].copy()
    df_sub.sort_values("Date", inplace=True)
    if len(df_sub)< window:
        return None
    df_sub[features]= df_sub[features].fillna(method="ffill").fillna(method="bfill")
    arr= scaler.transform(df_sub[features].iloc[-window:].values)
    return arr

def forecast_n_days_from_date(model, df_actual, start_date, window=35, horizon=31,
                              device=None, scaler=None, noise_std=0.1):
    """
    Full-blind forecast, each day appended with random noise
    to encourage fluctuations.
    """
    def add_noise_4d(pred_unscaled, std=0.05):
        # add random Gaussian noise to each dimension, clipped >0
        noise= np.random.normal(0.0, std, size=pred_unscaled.shape)
        return np.clip(pred_unscaled+ noise, a_min=0, a_max=None)

    arr_window= get_latest_window(df_actual, start_date, window, scaler)
    if arr_window is None:
        return pd.DataFrame()

    rolling_buffer= np.copy(arr_window)
    forecast_records=[]
    current_date= pd.to_datetime(start_date)

    for i in range(horizon):
        X_input= torch.tensor(rolling_buffer,dtype=torch.float).unsqueeze(0).to(device)
        with torch.no_grad():
            pred_scaled= model(X_input).cpu().numpy()[0]

        # invert scale
        pred_unscaled= scaler.inverse_transform(pred_scaled.reshape(1,-1))[0]
        # add noise => more fluctuation
        pred_noisy= add_noise_4d(pred_unscaled, noise_std)

        dayx= current_date+ pd.Timedelta(days=i)
        forecast_records.append({
            "ForecastDate": dayx,
            "Pred_Price_unscaled": pred_noisy[0],
            "Pred_Open_unscaled":  pred_noisy[1],
            "Pred_High_unscaled":  pred_noisy[2],
            "Pred_Low_unscaled":   pred_noisy[3]
        })

        # feed forward
        pred_noisy_scaled= scaler.transform(pred_noisy.reshape(1,-1))[0]
        rolling_buffer= np.vstack([rolling_buffer[1:], pred_noisy_scaled.reshape(1,-1)])

    return pd.DataFrame(forecast_records)

############################################
# 8A. Daily Plot
############################################
def plot_daily_two_charts(day_i, horizon_days, daily_fcst_dict, df_actual):
    df_sorted= df_actual.copy()
    df_sorted.sort_values("Date", inplace=True)
    day_end= day_i+ pd.Timedelta(days= horizon_days -1)

    mask= (df_sorted["Date"]<= day_end)
    df_plot= df_sorted.loc[mask].copy()

    # 1) Joint Diagram
    plt.figure(figsize=(12,5))
    plt.plot(df_plot["Date"], df_plot["Price"], color="black", label="Actual Price")

    color_map= {
        "LSTM": "red",
        "GRU": "blue",
        "RNN": "green",
        "CNN": "orange",
        "EnhancedLSTM": "magenta",
        "Transformer": "cyan",
        "N-BEATS": "brown",
        "N-HITS": "pink",
        "SVM": "olive",
        "GPR": "teal",
        "Boost": "purple"
    }
    for mname, fcdf in daily_fcst_dict.items():
        sub= fcdf[fcdf["ForecastDate"]<= day_end].copy()
        if not sub.empty:
            plt.plot(
                sub["ForecastDate"],
                sub["Pred_Price_unscaled"],
                color=color_map.get(mname,"gray"),
                linestyle="--",
                label=f"{mname} Forecast"
            )

    plt.title(f"Day {day_i.date()} - Joint Diagram (up to {day_end.date()})")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(prop={"size":8})
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 2) Zoom-In
    plt.figure(figsize=(12,5))
    maskz= (df_plot["Date"]>= day_i)&(df_plot["Date"]<= day_end)
    df_zoom= df_plot.loc[maskz].copy()
    if not df_zoom.empty:
        plt.plot(df_zoom["Date"], df_zoom["Price"], color="black", label="Actual Price")
    for mname, fcdf in daily_fcst_dict.items():
        sub= fcdf[(fcdf["ForecastDate"]>=day_i)&(fcdf["ForecastDate"]<=day_end)]
        if not sub.empty:
            plt.plot(
                sub["ForecastDate"],
                sub["Pred_Price_unscaled"],
                color=color_map.get(mname,"gray"),
                linestyle="--",
                label=f"{mname} Forecast"
            )

    plt.title(f"Day {day_i.date()} - Zoom-In Forecast ({day_i.date()} to {day_end.date()})")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(prop={"size":8})
    plt.grid(True)
    plt.tight_layout()
    plt.show()

############################################
# 8B. Rolling day-by-day
############################################
def rolling_train_validate_predict_moe(
    df_full,
    scaler_obj,
    model_types,
    start_train_cutoff=pd.to_datetime("2024-07-01"),
    start_val_cutoff=pd.to_datetime("2024-11-30"),
    start_pred=pd.to_datetime("2024-12-01"),
    end_pred=pd.to_datetime("2025-01-02"),
    horizon_days=30,
    do_daily_plots=True
):
    df_sorted= df_full.copy()
    df_sorted.sort_values("Date", inplace=True)

    df_scaled_all= df_sorted.copy()
    df_scaled_all[features]= df_scaled_all[features].fillna(method="ffill").fillna(method="bfill")
    df_scaled_all[features]= scaler_obj.transform(df_scaled_all[features])

    date_range= pd.date_range(start_pred, end_pred, freq="D")
    all_records= []

    for i,day_i in enumerate(date_range):
        train_cutoff_i= start_train_cutoff + pd.Timedelta(days=i)
        val_cutoff_i  = start_val_cutoff   + pd.Timedelta(days=i)

        print(f"\n=== Rolling Day {day_i.date()} ===")
        print(f"  Train cutoff={train_cutoff_i.date()}, Val cutoff={val_cutoff_i.date()}")
        print(f"  Forecast next {horizon_days} days from {day_i.date()} with noise")

        # 1) Train or load each model
        model_dict= {}
        for mt in model_types:
            out_file= f"best_{mt}_{day_i.strftime('%Y-%m-%d')}_V1"
            if mt in ["SVM","GPR","Boost"]:
                out_file += ".pkl"
                net= train_or_load_sklearn(
                    mt, df_scaled_all, train_cutoff_i, val_cutoff_i,
                    model_save_name=os.path.join(BEST_MODELS_DIR, out_file),
                    window_size=WINDOW_SIZE
                )
            else:
                out_file += ".pt"
                net= train_or_load_torch(
                    mt, WINDOW_SIZE, df_scaled_all, train_cutoff_i,val_cutoff_i,
                    model_save_name=os.path.join(BEST_MODELS_DIR, out_file),
                    num_epochs=NUM_EPOCHS
                )
            model_dict[mt]= net

        # 2) Forecast
        daily_fcst_dict= {}
        for mt, net in model_dict.items():
            df_fc= forecast_n_days_from_date(
                model= net,
                df_actual= df_sorted,  # unscaled
                start_date= day_i,
                window= WINDOW_SIZE,
                horizon= horizon_days,
                device= device,
                scaler= scaler_obj,
                noise_std=0.1 # random noise => more fluctuations
            )
            if not df_fc.empty:
                df_fc["BaseDate"]= day_i
                df_fc["Model"]= mt
                daily_fcst_dict[mt]= df_fc
                all_records.append(df_fc)
            else:
                daily_fcst_dict[mt]= pd.DataFrame()

        # see actual
        row_actual= df_sorted[df_sorted["Date"]== day_i]
        if not row_actual.empty:
            print(f"  Actual price: {row_actual['Price'].values[0]:.4f}")

        # 3) daily plots
        if do_daily_plots and daily_fcst_dict:
            plot_daily_two_charts(day_i, horizon_days, daily_fcst_dict, df_sorted)

    # Must do "concat" to combine DataFrames
    df_all = pd.concat(all_records, ignore_index=True) if all_records else pd.DataFrame()
    df_all.sort_values(["Model","BaseDate","ForecastDate"], inplace=True)
    return df_all

############################################
# 9. Evaluate & Plot
############################################
def walk_forward_test(model, df_all_scaled, test_start, window=35, scaler=None):
    dfw= df_all_scaled.copy()
    dfw.sort_values("Date", inplace=True)
    dates_test= np.sort(dfw[dfw["Date"]>=test_start]["Date"].unique())

    model.eval()
    preds=[]
    with torch.no_grad():
        for d in dates_test:
            idx= dfw.index[dfw["Date"]==d]
            if len(idx)==0:
                continue
            idx= idx[0]
            st= idx- window
            if st<0:
                continue
            window_data= dfw.iloc[st:idx][features].values
            if np.isnan(window_data).any():
                continue
            X_= torch.tensor(window_data, dtype=torch.float).unsqueeze(0).to(device)
            out_scaled= model(X_).cpu().numpy()[0]
            dfw.loc[idx, features]= out_scaled
            preds.append((d,*out_scaled))

    pred_df= pd.DataFrame(preds, columns=["Date","Pred_Price_scaled","Pred_Open_scaled","Pred_High_scaled","Pred_Low_scaled"])
    arr_s= pred_df[["Pred_Price_scaled","Pred_Open_scaled","Pred_High_scaled","Pred_Low_scaled"]].values
    arr_un= scaler.inverse_transform(arr_s)
    pred_df["Pred_Price_unscaled"]= arr_un[:,0]
    pred_df["Pred_Open_unscaled"]=  arr_un[:,1]
    pred_df["Pred_High_unscaled"]= arr_un[:,2]
    pred_df["Pred_Low_unscaled"]=  arr_un[:,3]
    return pred_df.sort_values("Date")

def evaluate_and_plot_all_models(results_dict, df_scaled, df_train, 
                                 train_cutoff, val_cutoff, test_cutoff, 
                                 csv_file_complete="McDonald’s Stock Price History_Complete.csv"):
    df_real= df_scaled.copy()
    df_real.sort_values("Date", inplace=True)
    mask_tv= (df_real["Date"]<test_cutoff)
    df_tv= df_real[mask_tv].copy()
    arr_s= df_tv[features].values
    from sklearn.preprocessing import MinMaxScaler
    arr_un= scaler.inverse_transform(arr_s)
    df_tv["Real_Price_unscaled"]= arr_un[:,0]

    plt.figure(figsize=(14,7))
    plt.plot(df_tv["Date"], df_tv["Real_Price_unscaled"], color="black", label="Actual(Train+Val)")

    merged_df= df_real.copy()
    merged_df.sort_values("Date", inplace=True)

    color_map= {
        "LSTM": "red",
        "GRU": "blue",
        "RNN": "green",
        "CNN": "orange",
        "EnhancedLSTM": "magenta",
        "Transformer": "cyan",
        "N-BEATS": "brown",
        "N-HITS": "pink",
        "SVM": "olive",
        "GPR": "teal",
        "Boost": "purple"
    }

    for mname,infodict in results_dict.items():
        val_df= infodict.get("val_pred_df", None)
        test_df= infodict.get("test_pred_df", None)
        frames=[]
        if val_df is not None:
            frames.append(val_df.rename(columns={"Pred_Price_unscaled":f"Pred_{mname}"}))
        if test_df is not None:
            frames.append(test_df.rename(columns={"Pred_Price_unscaled":f"Pred_{mname}"}))
        if frames:
            comb= pd.concat(frames, ignore_index=True)
            comb= comb[["Date",f"Pred_{mname}"]].drop_duplicates("Date")
            comb.sort_values("Date", inplace=True)
            merged_df= pd.merge(merged_df, comb, on="Date", how="left")

    for mname in results_dict.keys():
        col= f"Pred_{mname}"
        if col in merged_df.columns:
            plt.plot(
                merged_df["Date"],
                merged_df[col],
                color=color_map.get(mname,"gray"),
                linestyle="-",
                label=f"{mname} Pred"
            )

    plt.axvspan(df_train["Date"].min(), train_cutoff, color="skyblue", alpha=0.1, label="Train")
    plt.axvspan(train_cutoff, val_cutoff, color="green", alpha=0.1, label="Validation")
    plt.axvspan(test_cutoff, merged_df["Date"].max(), color="yellow", alpha=0.1, label="Test")

    plt.title("Joint Diagram: Actual Price vs. Predictions")
    plt.xlabel("Date")
    plt.ylabel("Price(unscaled)")
    plt.legend(prop={"size":8})
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Zoom Test
    df_test_zoom= merged_df[merged_df["Date"]>= test_cutoff]
    if not df_test_zoom.empty:
        plt.figure(figsize=(12,5))
        for mname in results_dict.keys():
            c= f"Pred_{mname}"
            if c in df_test_zoom.columns:
                plt.plot(df_test_zoom["Date"], df_test_zoom[c], linestyle="--", label=f"{mname} Pred")
        # overlay actual
        df_complete= pd.read_csv(csv_file_complete)
        df_complete["Date"]= pd.to_datetime(df_complete["Date"], errors="coerce")
        for drop_c in ["Vol.","Change %"]:
            if drop_c in df_complete.columns:
                df_complete.drop(columns=[drop_c], inplace=True, errors="ignore")
        df_complete.sort_values("Date", inplace=True)
        for f in features:
            df_complete[f]= df_complete[f].astype(str).str.replace(",","",regex=True).astype(float)
        df_test_only= df_complete[df_complete["Date"]>= test_cutoff].copy()
        plt.plot(df_test_only["Date"], df_test_only["Price"], color="black",lw=2, label="ActualTestPrice")

        plt.title("Test Period Zoom-In")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend(prop={"size":8})
        plt.grid(True)
        plt.tight_layout()
        plt.show()

############################################
# 10. Final "concatenated" rolling forecasts
############################################
def plot_final_rolling_fc(final_rolling_fcst, df_actual,
                          from_d, to_d):
    if final_rolling_fcst.empty:
        print("No final rolling forecast data. Exiting.")
        return

    df_act= df_actual.copy()
    df_act.sort_values("Date", inplace=True)
    mask= (df_act["Date"]>= from_d)&(df_act["Date"]<= to_d)
    df_a= df_act[mask].copy()

    plt.figure(figsize=(12,5))
    plt.plot(df_a["Date"], df_a["Price"], color="black", label="ActualPrice")

    color_map= {
        "LSTM": "red",
        "GRU": "blue",
        "RNN": "green",
        "CNN": "orange",
        "EnhancedLSTM": "magenta",
        "Transformer": "cyan",
        "N-BEATS": "brown",
        "N-HITS": "pink",
        "SVM": "olive",
        "GPR": "teal",
        "Boost": "purple"
    }
    for m in final_rolling_fcst["Model"].unique():
        df_m= final_rolling_fcst[final_rolling_fcst["Model"]== m].copy()
        df_m= df_m[(df_m["ForecastDate"]>=from_d)&(df_m["ForecastDate"]<= to_d)]
        if not df_m.empty:
            plt.plot(df_m["ForecastDate"], df_m["Pred_Price_unscaled"],
                     color=color_map.get(m,"gray"), linestyle="--",label=f"{m} Forecast")

    plt.title(f"Rolling Forecast from {from_d.date()} to {to_d.date()}")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(prop={"size":8})
    plt.grid(True)
    plt.tight_layout()
    plt.show()

############################################
# MAIN
############################################
if __name__=="__main__":
    # 1) Load Data
    df_scaled, df_complete_all, scaler= load_and_scale_data(
        "McDonald’s Stock Price History.csv",
        "McDonald’s Stock Price History_Complete.csv"
    )

    # 2) Example: train or load static
    # ...
    # 3) Rolling approach
    final_rolling_fc= rolling_train_validate_predict_moe(
        df_full= df_complete_all,
        scaler_obj= scaler,
        model_types= ["LSTM","GRU","RNN","CNN","Transformer","N-BEATS","N-HITS","SVM","Boost"],
        start_train_cutoff=pd.to_datetime("2024-07-01"),
        start_val_cutoff=pd.to_datetime("2024-11-30"),
        start_pred=pd.to_datetime("2024-12-01"),
        end_pred=pd.to_datetime("2025-01-02"),
        horizon_days=30,
        do_daily_plots=True
    )

    # 4) Plot final
    print("===== ROLLING FC HEAD =====")
    print(final_rolling_fc.head(50))
    plot_final_rolling_fc(final_rolling_fc, df_complete_all, 
                          from_d=pd.to_datetime("2024-12-01"),
                          to_d=pd.to_datetime("2025-01-31"))
